In [9]:
%%bash
conda env list # check environments
conda list | sort | grep 'nltk\|spacy\|gensim\|pandas\|numpy\|scikit\|sklearn\|keras\|tensor' # check packages

# conda environments:
#
base                     /home/frass/anaconda3
keras                 *  /home/frass/anaconda3/envs/keras
keras-gpu                /home/frass/anaconda3/envs/keras-gpu
tf_gpu                   /home/frass/anaconda3/envs/tf_gpu

# packages in environment at /home/frass/anaconda3/envs/keras:
gensim                    3.8.0            py37h962f231_0  
keras                     2.2.4                         0  
keras-applications        1.0.8                      py_0  
keras-base                2.2.4                    py37_0  
keras-preprocessing       1.1.0                      py_1  
nltk                      3.4.5                    py37_0  
numpy                     1.17.3           py37h95a1406_0    conda-forge
pandas                    0.25.3           py37he6710b0_0  
scikit-learn              0.21.3           py37h22eb022_0  
spacy                     2.2.3            py37hc9558a2_0    conda-forge
tensorboard               1.15.0             pyhb230dea_0  


In [42]:
%%bash
python -m spacy download en_core_web_lg

  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp37-none-any.whl size=829180945 sha256=128ac62fa082178f69e1074a7bdce854cca5be3e1e183cadf40086ec45181196
  Stored in directory: /tmp/pip-ephem-wheel-cache-i1_wxsob/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [1]:
import gensim
import nltk
import sklearn
import pandas as pd
import keras
import csv
import time
from nltk.tokenize import TreebankWordTokenizer
import nltk, re, pprint
from nltk import word_tokenize

Using TensorFlow backend.


In [2]:
DATA_PATH = 'Articles.csv'
ENCODING = "utf-8"
MODEL_PATH = '~/media/D/'
TITLE_PATH = 'titles.txt'
ARTICLE_PATH = 'articles.txt'

In [3]:
corpus = pd.read_csv(DATA_PATH, encoding=ENCODING)
corpus.columns

Index(['Unnamed: 0', 'id', 'title', 'text'], dtype='object')

In [4]:
articles = [str(text) for text in corpus.text]
titles = [str(title) for title in corpus.title]
# with open(TITLE_PATH, 'w') as outfile:
#     for line in titles:
#         outfile.write(line.)
# with open(ARTICLE_PATH, 'w') as outfile:
#     for line in articles:
#         outfile.write(line)
                

In [5]:
corpus.shape
        

(197377, 4)

In [242]:
import re
# nonword = re.compile(r'^(\W*|[0-9]*)$')   # to remove numbers
nonword = re.compile(r'^\W*$') # to keep numbers

In [243]:
not nonword.search('gwwsssz')

True

In [13]:
nltk.download('stopwords')
nltk.download('punkt')
stopwords = nltk.corpus.stopwords.words('english')
stopwords += '- -- " \' ? , . ! * ** *** ( ) = == === : ; \'\' ` `` [ ] & %'.split()


[nltk_data] Downloading package stopwords to /home/frass/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/frass/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
australia = corpus.loc[13]
australiaArticle = australia.text

In [117]:
def tokenize(article, tokenizer=TreebankWordTokenizer()):
    article_tokens = [tok for tok in tokenizer.tokenize(article) if (tok.lower() not in stopwords and not nonword.search(tok))]
    return article_tokens

In [118]:
t = time.time()
my_tokens = [tokenize(str(art)) for art in corpus.text]
print(time.time()-t)

143.016672372818


In [119]:
import nltk, re, pprint
from nltk import word_tokenize


In [37]:
# t = time.time()
# my_tokens = [word_tokenize(str(art)) for art in corpus.text]
# print(time.time()-t)

175.67496347427368


In [120]:
from collections import defaultdict

In [121]:
frequency = defaultdict(int)
for article in my_tokens:
    for tok in article:
        frequency[tok] += 1
my_tokens = [[tok for tok in article if frequency[tok] > 5]
    for article in my_tokens]

In [87]:
def nltk_vectorize(article):
    features = defaultdict(int)
    for tok in article:
        features[tok] += 1
    return features

In [92]:
from nltk.text import TextCollection
def nltk_tfidf_vectorize(lists_of_tokens):
    texts = TextCollection(lists_of_tokens)
    for article in lists_of_tokens:
        yield {
            term: texts.tf_idf(term, article)
            for term in article
        }
    

In [93]:
nltk_tfidf_vectors = nltk_tfidf_vectorize(my_tokens)

In [89]:
nltk_vectors = map(nltk_vectorize, my_tokens)

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(my_tokens)
dictionary.save('/tmp/simplewiki_mystopwords.dict')  # store the dictionary, for future reference

In [123]:
print(dictionary)

Dictionary(149065 unique tokens: ["'M", "'s", '*April', '1', '1.']...)


In [71]:
help(dictionary)

Help on Dictionary in module gensim.corpora.dictionary object:

class Dictionary(gensim.utils.SaveLoad, collections.abc.Mapping)
 |  Dictionary(documents=None, prune_at=2000000)
 |  
 |  Dictionary encapsulates the mapping between normalized words and their integer ids.
 |  
 |  Notable instance attributes:
 |  
 |  Attributes
 |  ----------
 |  token2id : dict of (str, int)
 |      token -> tokenId.
 |  id2token : dict of (int, str)
 |      Reverse mapping for token2id, initialized in a lazy manner to save memory (not created until needed).
 |  cfs : dict of (int, int)
 |      Collection frequencies: token_id -> how many instances of this token are contained in the documents.
 |  dfs : dict of (int, int)
 |      Document frequencies: token_id -> how many documents contain this token.
 |  num_docs : int
 |      Number of documents processed.
 |  num_pos : int
 |      Total number of corpus positions (number of processed words).
 |  num_nnz : int
 |      Total number of non-zeroes in th

In [77]:
dictionary.num_nnz

11161994

In [ ]:
def vectorize(doc):
    features = defaultdict(int)
    for token in tokenize(doc):
        features[token] += 1
    return features

nltk_vectors = map(vectorize, corpus)

In [124]:
gensim_vectors = [
    dictionary.doc2bow(doc) for doc in my_tokens
]

In [125]:
gensim_tfidf = gensim.models.TfidfModel(dictionary=dictionary, normalize=True)
gensim_tfidf_vectors = [gensim_tfidf[dictionary.doc2bow(article)] for article in my_tokens]

In [126]:
dictionary.save_as_text('gensim_dictionary.txt', sort_by_word=True)
gensim_tfidf.save('gensim_tfidf.pkl')

2019-12-06 16:52:28,459 : INFO : saving dictionary mapping to gensim_dictionary.txt
2019-12-06 16:52:28,804 : INFO : saving TfidfModel object under gensim_tfidf.pkl, separately None
2019-12-06 16:52:29,389 : INFO : saved gensim_tfidf.pkl


In [ ]:
# dictionary = gensim.corpora.Dictionary.load_from_text('gensim_dictionary.txt')
# gensim_tfidf = gensim.models.TfidfModel.load('gensim_tfidf.pkl')

In [152]:
lexicon = set()
for article in my_tokens:
    lexicon.update(article)
len(lexicon)

149065

In [127]:
from sklearn.feature_extraction.text import TfidfVectorizer

sklearn_tfidf  = TfidfVectorizer(input=articles, encoding=ENCODING,decode_error='replace' '<UNK>', stop_words=stopwords, min_df=0.00005)
sklearn_corpus = sklearn_tfidf.fit_transform(articles)

In [128]:
sklearn_corpus.shape

(197377, 55993)

In [37]:
# sklearn_corpus[13].todense()

matrix([[0.        , 0.02413215, 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [50]:
# corpus[corpus['title'] == 'Dog']

,Unnamed: 0,id,title,text
953,1285,954,Dog,\r\n\r\n\r\n'''Dogs''' (''Canis lupus familiar...


In [30]:
# corpus.loc[1554]

Unnamed: 0                                                 2079
id                                                         1555
title                                               New Zealand
text          \r\n\r\n'''New Zealand''' (also known as '''''...
Name: 1554, dtype: object

In [39]:
from scipy import spatial
def cos_sim(ind1, ind2):
    return 1 - spatial.distance.cosine(sklearn_corpus[ind1].todense(), sklearn_corpus[ind2].todense())

In [129]:
cos_sim(616, 953)

0.11973753682475463

In [210]:
from gensim import models
NUM_TOPICS = 500
lsi_model = models.LsiModel(gensim_tfidf_vectors, id2word=dictionary, num_topics=NUM_TOPICS)
corpus_lsi = lsi_model[gensim_tfidf_vectors]

2019-12-06 17:41:48,447 : INFO : using serial LSI version on this node
2019-12-06 17:41:48,448 : INFO : updating model with new documents
2019-12-06 17:41:48,449 : INFO : preparing a new chunk of documents
2019-12-06 17:41:48,905 : INFO : using 100 extra samples and 2 power iterations
2019-12-06 17:41:48,905 : INFO : 1st phase: constructing (149065, 600) action matrix
2019-12-06 17:41:50,822 : INFO : orthonormalizing (149065, 600) action matrix
2019-12-06 17:42:39,175 : INFO : 2nd phase: running dense svd on (600, 20000) matrix
2019-12-06 17:42:41,826 : INFO : computing the final decomposition
2019-12-06 17:42:41,827 : INFO : keeping 500 factors (discarding 6.200% of energy spectrum)
2019-12-06 17:42:43,079 : INFO : processed documents up to #20000
2019-12-06 17:42:43,084 : INFO : topic #0(18.437): 0.595*"b." + 0.591*"d." + 0.247*"American" + 0.109*"footballer" + 0.106*"politician" + 0.103*"actor" + 0.091*"actress" + 0.082*"English" + 0.079*"German" + 0.078*"writer"
2019-12-06 17:42:43

2019-12-06 17:47:16,351 : INFO : merging projections: (149065, 500) + (149065, 500)
2019-12-06 17:47:31,797 : INFO : keeping 500 factors (discarding 6.364% of energy spectrum)
2019-12-06 17:47:34,057 : INFO : processed documents up to #100000
2019-12-06 17:47:34,063 : INFO : topic #0(52.019): 0.737*"France" + 0.284*"Communes" + 0.275*"Regions" + 0.266*"Departments" + 0.262*"France." + 0.200*"found" + 0.123*"north" + 0.108*"Pas-de-Calais" + 0.103*"Nord-Pas-de-Calais" + 0.095*"Aisne"
2019-12-06 17:47:34,068 : INFO : topic #1(48.939): -1.000*"nan" + -0.000*"island" + -0.000*"Lewis" + -0.000*"Harris" + -0.000*"Hebrides" + -0.000*"Isles" + -0.000*"Loch" + -0.000*"Gaelic" + -0.000*"Clan" + -0.000*"Scottish"
2019-12-06 17:47:34,072 : INFO : topic #2(27.880): 0.527*"2" + 0.327*"League" + 0.323*"Division" + 0.251*"football" + 0.218*"statistics==" + 0.196*"career" + 0.195*"|J." + 0.174*"Association" + 0.168*"player." + 0.158*"==Club"
2019-12-06 17:47:34,078 : INFO : topic #3(24.790): 0.544*"city

2019-12-06 17:52:29,153 : INFO : topic #1(56.585): 0.737*"France" + 0.308*"Communes" + 0.294*"Departments" + 0.285*"France." + 0.237*"Regions" + 0.203*"found" + 0.108*"north" + 0.091*"Pas-de-Calais" + 0.086*"Nord-Pas-de-Calais" + 0.082*"department"
2019-12-06 17:52:29,157 : INFO : topic #2(30.939): 0.288*"United" + 0.270*"city" + 0.269*"States" + 0.240*"2" + 0.167*"Iowa" + 0.153*"League" + 0.142*"football" + 0.132*"Division" + 0.126*"born" + 0.113*"County"
2019-12-06 17:52:29,161 : INFO : topic #3(27.858): -0.420*"2" + 0.324*"city" + 0.275*"States" + -0.265*"League" + 0.262*"United" + -0.257*"Division" + 0.232*"Iowa" + -0.214*"football" + -0.173*"statistics==" + -0.157*"career"
2019-12-06 17:52:29,166 : INFO : topic #4(27.203): -0.553*"district" + -0.521*"Switzerland" + -0.378*"municipality" + -0.314*"Cantons" + -0.193*"canton" + -0.150*"administrative" + 0.115*"States" + -0.115*"Switzerland." + 0.115*"United" + 0.108*"city"
2019-12-06 17:52:29,168 : INFO : preparing a new chunk of doc

In [131]:
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [211]:
from gensim.test.utils import get_tmpfile
tmp_fname = get_tmpfile("simple_wiki_lsi_{}.model".format(NUM_TOPICS))
lsi_model.save(tmp_fname) 


2019-12-06 17:53:42,050 : INFO : saving Projection object under /tmp/simple_wiki_lsi_500.model.projection, separately None
2019-12-06 17:53:42,051 : INFO : storing np array 'u' to /tmp/simple_wiki_lsi_500.model.projection.u.npy
2019-12-06 17:53:42,321 : INFO : saved /tmp/simple_wiki_lsi_500.model.projection
2019-12-06 17:53:42,321 : INFO : saving LsiModel object under /tmp/simple_wiki_lsi_500.model, separately None
2019-12-06 17:53:42,322 : INFO : not storing attribute projection
2019-12-06 17:53:42,322 : INFO : not storing attribute dispatcher
2019-12-06 17:53:42,377 : INFO : saved /tmp/simple_wiki_lsi_500.model


In [ ]:
# loaded_model = LsiModel.load(tmp_fname)

In [212]:
lsi_model.print_debug(num_topics=5, num_words=10)

2019-12-06 17:53:42,380 : INFO : computing word-topic salience for 5 topics
2019-12-06 17:53:44,566 : INFO : topic #0(94.953): nan(-1.000), ..., 
2019-12-06 17:53:44,733 : INFO : topic #1(56.868): France(0.737), Departments(0.296), Communes(0.309), Regions(0.235), France.(0.286), found(0.202), north(0.107), Nord-Pas-de-Calais(0.085), Pas-de-Calais(0.090), Picardie(0.078), ..., 
2019-12-06 17:53:44,872 : INFO : topic #2(34.016): United(0.372), States(0.358), city(0.293), County(0.282), seat(0.129), state(0.162), county(0.161), Iowa(0.164), U.S.(0.137), 2010(0.129), ..., Departments(-0.015), Communes(-0.012), Regions(-0.012), France(-0.015), Nord-Pas-de-Calais(-0.006)
2019-12-06 17:53:44,970 : INFO : topic #3(32.453): return(0.998), File-Date(0.002), map/data/Canada(0.001), DEU(0.000), func(0.000), GBR(0.000), mw.text.unstripNoWiki(0.000), iter(0.000), title.namespace(0.000), snippets.username(0.000), ..., United(-0.018), States(-0.018), County(-0.015), city(-0.015), seat(-0.007)
2019-12

In [213]:
lsi_model.show_topic(41)

[('UTC', -0.47021990260401497),
 ('Illinois', 0.31999465835741614),
 ('Kentucky', -0.17973182676232607),
 ('game', -0.16661211150833988),
 ('movie', -0.15172156746873144),
 ('New', -0.14660716989758735),
 ('series', 0.14076637580630477),
 ('television', 0.11626929593465471),
 ('WWE', 0.11057377755786744),
 ('page', -0.1096112022768201)]

In [214]:
from gensim import similarities
from gensim.test.utils import datapath, get_tmpfile
index_temp = get_tmpfile("index")
index = similarities.Similarity(index_temp, gensim_vectors, num_features=len(dictionary.keys()))  # transform corpus to LSI space and index it



2019-12-06 17:53:45,190 : INFO : starting similarity index under /tmp/index
2019-12-06 17:53:46,739 : INFO : PROGRESS: fresh_shard size=10000
2019-12-06 17:53:48,089 : INFO : PROGRESS: fresh_shard size=20000
2019-12-06 17:53:49,424 : INFO : PROGRESS: fresh_shard size=30000
2019-12-06 17:53:49,789 : INFO : creating sparse index
2019-12-06 17:53:49,789 : INFO : creating sparse matrix from corpus
2019-12-06 17:53:49,791 : INFO : PROGRESS: at document #0/32768
2019-12-06 17:53:52,036 : INFO : PROGRESS: at document #10000/32768
2019-12-06 17:53:53,194 : INFO : PROGRESS: at document #20000/32768
2019-12-06 17:53:54,233 : INFO : PROGRESS: at document #30000/32768
2019-12-06 17:53:54,470 : INFO : created <32768x149065 sparse matrix of type '<class 'numpy.float32'>'
	with 3255422 stored elements in Compressed Sparse Row format>
2019-12-06 17:53:54,471 : INFO : creating sparse shard #0
2019-12-06 17:53:54,471 : INFO : saving index shard to /tmp/index.0
2019-12-06 17:53:54,471 : INFO : saving Spa

list

In [216]:
query_vec = "Image compression is a type of data compression applied to digital\
images, to reduce their cost for storage or transmission. Algorithms may take\
advantage of visual perception and the statistical properties of image data to\
provide superior results compared with generic data compression methods which\
are used for other digital data."

In [244]:
query_vec = "Microsoft Corporation is an American multinational technology company with headquarters in Redmond, Washington. It develops, manufactures, licenses, supports, and sells computer software, consumer electronics, personal computers, and related services. Its best known software products are the Microsoft Windows line of operating systems, the Microsoft Office suite, and the Internet Explorer and Edge web browsers. Its flagship hardware products are the Xbox video game consoles and the Microsoft Surface lineup of touchscreen personal computers"

In [245]:
vec_bow = dictionary.doc2bow(tokenize(query_vec))
vec_lsi = lsi_model[vec_bow]
sims = index[vec_lsi]  # perform a similarity query against the corpus
for i, s in sorted(enumerate(sims), key=lambda item: -item[1])[:10]:
    print(s, i, titles[i], "\n\t", articles[i][:75])

0.10887238 134580 2014 Asian Games 
	 

The '''2014 Asian Games''', officially known as the '''XVII Asiad''', w
0.10415807 182760 Last characters in Waterloo Road (7, 8, 9, 10) 
	 This is a list of pupils and teachers who were in Series' 7, 8, 9 and 10 of
0.102456704 185874 Frederick Terman 
	 

'''Frederick Emmons Terman''' (; June 7, 1900 – December 19, 1982) was 
0.10152025 124149 Albert I of Belgium 
	 

'''Albert I''' (April 8, 1875 – February 17, 1934) reigned as List of B
0.10019777 193908 Ernst Brenner 
	 
'''Ernst Brenner''' (9 December 1856 – 11 March 1911) was a Switzerland p
0.098908655 125204 Georgina Henry 
	 '''Georgina Clare Henry''' (8 June 1960 – 7 February 2014)Alan Rushbridger 
0.09798015 170284 Elaine H. Kim 
	 '''Elaine H. Kim'''  (February 26, 1972 - Present) teaches Asian American S
0.09742902 79244 Daryn Colledge 
	 
'''Daryn Colledge''' (born February 11, 1982 in Fairbanks, Alaska) is an 
0.097250484 3847 1927 
	 

== Events ==
* The Hardy Boys mystery series 

In [220]:
sum(['Category:' not in title for title in titles])

155581

In [246]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=gensim_vectors, id2word=dictionary, num_topics=NUM_TOPICS, passes=1)
lda_model.print_topics(10)


2019-12-06 18:12:49,994 : INFO : using symmetric alpha at 0.002
2019-12-06 18:12:49,994 : INFO : using symmetric eta at 0.002
2019-12-06 18:12:50,008 : INFO : using serial LDA version on this node
2019-12-06 18:12:56,128 : INFO : running online (single-pass) LDA training, 500 topics, 1 passes over the supplied corpus of 197377 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2019-12-06 18:12:56,129 : INFO : PROGRESS: pass 0, at document #2000/197377
2019-12-06 18:13:10,421 : INFO : merging changes from 2000 documents into a model of 197377 documents
2019-12-06 18:13:13,498 : INFO : topic #429 (0.002): 0.011*"people" + 0.008*"also" + 0.007*"language" + 0.006*"many" + 0.006*"'s" + 0.005*"one" + 0.005*"Google" + 0.004*"called" + 0.004*"Chinese" + 0.004*"used"
2019-12-06 18:13:13,500 : INFO : topic #30 (0.002): 0.012*"people" + 0.008*"language" + 0.006*"also" + 0.005*"used" + 0.005*"ghos

2019-12-06 18:14:23,107 : INFO : topic diff=2.080128, rho=0.408248
2019-12-06 18:14:23,125 : INFO : PROGRESS: pass 0, at document #14000/197377
2019-12-06 18:14:31,995 : INFO : merging changes from 2000 documents into a model of 197377 documents
2019-12-06 18:14:35,112 : INFO : topic #205 (0.002): 0.165*"till" + 0.049*"shift" + 0.040*"gray" + 0.027*"justify" + 0.025*"Big" + 0.024*"driver" + 0.023*"align" + 0.020*"ImageSize" + 0.020*"AlignBars" + 0.020*"DateFormat"
2019-12-06 18:14:35,113 : INFO : topic #384 (0.002): 0.061*"ship" + 0.053*"ships" + 0.040*"whale" + 0.014*"would" + 0.012*"twice" + 0.011*"whaling" + 0.011*"boats" + 0.010*"around" + 0.010*"'s" + 0.009*"boat"
2019-12-06 18:14:35,114 : INFO : topic #446 (0.002): 0.260*"century" + 0.071*"20th" + 0.056*"year" + 0.053*"19th" + 0.033*"first" + 0.023*"Note" + 0.023*"years" + 0.022*"18th" + 0.019*"last" + 0.018*"17th"
2019-12-06 18:14:35,116 : INFO : topic #284 (0.002): 0.044*"palm" + 0.017*"witches" + 0.015*"microscope" + 0.014*"18

2019-12-06 18:15:43,967 : INFO : topic diff=2.477957, rho=0.288675
2019-12-06 18:15:43,985 : INFO : PROGRESS: pass 0, at document #26000/197377
2019-12-06 18:15:51,753 : INFO : merging changes from 2000 documents into a model of 197377 documents
2019-12-06 18:15:54,857 : INFO : topic #193 (0.002): 0.126*"Green" + 0.057*"Jordan" + 0.050*"Claudius" + 0.050*"mixing" + 0.038*"no." + 0.023*"Syrian" + 0.021*"Syria" + 0.018*"cone" + 0.018*"Solomon" + 0.016*"shade"
2019-12-06 18:15:54,858 : INFO : topic #143 (0.002): 0.034*"Society" + 0.027*"S." + 0.024*"E." + 0.023*"J." + 0.022*"Journal" + 0.022*"pp." + 0.022*"C." + 0.021*"H." + 0.018*"A." + 0.017*"M."
2019-12-06 18:15:54,860 : INFO : topic #235 (0.002): 0.113*"state" + 0.037*"states" + 0.034*"State" + 0.016*"state." + 0.014*"independent" + 0.013*"government" + 0.013*"territory" + 0.013*"Native" + 0.010*"declared" + 0.010*"lands"
2019-12-06 18:15:54,861 : INFO : topic #469 (0.002): 0.054*"Edward" + 0.051*"England" + 0.048*"King" + 0.044*"Henr

2019-12-06 18:16:53,746 : INFO : merging changes from 2000 documents into a model of 197377 documents
2019-12-06 18:16:56,846 : INFO : topic #445 (0.002): 0.104*"primarily" + 0.078*"watch" + 0.047*"serpent" + 0.042*"Comet" + 0.041*"synonym" + 0.028*"favorable" + 0.027*"*Rock" + 0.025*"resin" + 0.025*"artwork" + 0.024*"name=="
2019-12-06 18:16:56,847 : INFO : topic #277 (0.002): 0.035*"insects" + 0.025*"flight" + 0.021*"wings" + 0.018*"stages" + 0.017*"insect" + 0.016*"species" + 0.015*"biology" + 0.014*"fly" + 0.013*"wing" + 0.012*"eggs"
2019-12-06 18:16:56,849 : INFO : topic #89 (0.002): 0.118*"nuclear" + 0.050*"Nuclear" + 0.042*"element" + 0.032*"elements" + 0.026*"atomic" + 0.022*"nucleus" + 0.018*"atom" + 0.018*"stable" + 0.018*"decay" + 0.016*"atoms"
2019-12-06 18:16:56,850 : INFO : topic #388 (0.002): 0.078*"God" + 0.039*"believe" + 0.027*"also" + 0.026*"Buddhism" + 0.023*"one" + 0.018*"belief" + 0.016*"called" + 0.013*"many" + 0.012*"Spirit" + 0.011*"God."
2019-12-06 18:16:56,85

2019-12-06 18:17:56,097 : INFO : PROGRESS: pass 0, at document #50000/197377
2019-12-06 18:18:00,438 : INFO : merging changes from 2000 documents into a model of 197377 documents
2019-12-06 18:18:03,534 : INFO : topic #195 (0.002): 0.135*"built" + 0.058*"building" + 0.046*"buildings" + 0.022*"tallest" + 0.020*"construction" + 0.016*"world" + 0.015*"Tower" + 0.014*"one" + 0.014*"build" + 0.013*"designed"
2019-12-06 18:18:03,535 : INFO : topic #301 (0.002): 0.073*"UTC" + 0.039*"n't" + 0.027*"2008" + 0.026*"page" + 0.016*"would" + 0.016*"think" + 0.015*"articles" + 0.014*"Wikipedia" + 0.013*"'s" + 0.013*"article"
2019-12-06 18:18:03,537 : INFO : topic #397 (0.002): 0.213*"US" + 0.073*"per" + 0.030*"rate" + 0.028*"product" + 0.027*"economy" + 0.026*"domestic" + 0.026*"Kim" + 0.024*"income" + 0.019*"age" + 0.017*"costs"
2019-12-06 18:18:03,538 : INFO : topic #341 (0.002): 0.197*"Generation" + 0.096*"programme" + 0.054*"Seal" + 0.053*"tobacco" + 0.044*"Gene" + 0.039*"smoking" + 0.030*"someth

...

2019-12-06 18:27:09,404 : INFO : topic diff=0.040727, rho=0.113228
2019-12-06 18:27:09,422 : INFO : PROGRESS: pass 0, at document #158000/197377
2019-12-06 18:27:15,178 : INFO : merging changes from 2000 documents into a model of 197377 documents
2019-12-06 18:27:18,265 : INFO : topic #189 (0.002): 0.220*"Boston" + 0.201*"Massachusetts" + 0.054*"Massachusetts." + 0.047*"Rhode" + 0.046*"Vermont" + 0.036*"Mayflower" + 0.028*"Swift" + 0.028*"Plymouth" + 0.026*"Colony" + 0.024*"Miles"
2019-12-06 18:27:18,267 : INFO : topic #337 (0.002): 0.107*"citizenship" + 0.090*"Engineer" + 0.085*"el" + 0.077*"Riley" + 0.068*"Sisters" + 0.048*"Elliot" + 0.046*"Flores" + 0.034*"dei" + 0.033*"skater" + 0.033*"Valle"
2019-12-06 18:27:18,268 : INFO : topic #13 (0.002): 0.360*"crashes" + 0.080*"metal" + 0.056*"break" + 0.030*"breaks" + 0.029*"broken" + 0.026*"heavy" + 0.025*"Metal" + 0.019*"bolt" + 0.018*"phase" + 0.017*"focuses"
2019-12-06 18:27:18,270 : INFO : topic #432 (0.002): 0.246*"2012." + 0.172*"One

2019-12-06 18:28:11,352 : INFO : topic diff=0.036801, rho=0.109109
2019-12-06 18:28:11,371 : INFO : PROGRESS: pass 0, at document #170000/197377
2019-12-06 18:28:15,882 : INFO : merging changes from 2000 documents into a model of 197377 documents
2019-12-06 18:28:18,973 : INFO : topic #55 (0.002): 0.455*"South" + 0.103*"America" + 0.090*"Central" + 0.035*"America." + 0.027*"Colombia" + 0.021*"Uruguay" + 0.017*"Venezuela" + 0.016*"Paraguay" + 0.016*"Ecuador" + 0.012*"resort"
2019-12-06 18:28:18,974 : INFO : topic #72 (0.002): 0.351*"International" + 0.175*"Airport" + 0.139*"Airlines" + 0.040*"airport" + 0.032*"airline" + 0.021*"Airport." + 0.020*"Delta" + 0.020*"Lines" + 0.017*"1980." + 0.017*"Air"
2019-12-06 18:28:18,976 : INFO : topic #183 (0.002): 0.471*"New" + 0.217*"York" + 0.091*"Jersey" + 0.050*"City" + 0.034*"Times" + 0.027*"American" + 0.015*"Connecticut" + 0.010*"Hampshire" + 0.006*"Minneapolis" + 0.005*"MO"
2019-12-06 18:28:18,977 : INFO : topic #226 (0.002): 0.209*"Roman" + 

2019-12-06 18:29:07,431 : INFO : topic diff=0.034308, rho=0.105409
2019-12-06 18:29:07,449 : INFO : PROGRESS: pass 0, at document #182000/197377
2019-12-06 18:29:12,483 : INFO : merging changes from 2000 documents into a model of 197377 documents
2019-12-06 18:29:15,584 : INFO : topic #68 (0.002): 0.166*"car" + 0.061*"produced" + 0.042*"drive" + 0.040*"driver" + 0.033*"Motor" + 0.033*"fuel" + 0.032*"vehicles" + 0.030*"truck" + 0.027*"producing" + 0.023*"Car"
2019-12-06 18:29:15,586 : INFO : topic #179 (0.002): 0.103*"engine" + 0.099*"partner" + 0.061*"firm" + 0.056*"loop" + 0.034*"steam" + 0.030*"engines" + 0.030*"partnership" + 0.028*"tube" + 0.027*"delivered" + 0.026*"shot."
2019-12-06 18:29:15,587 : INFO : topic #341 (0.002): 0.110*"physician" + 0.075*"programme" + 0.070*"Gloria" + 0.063*"Generation" + 0.058*"Gene" + 0.049*"tobacco" + 0.047*"bringing" + 0.041*"Garrett" + 0.041*"Sally" + 0.032*"Academy."
2019-12-06 18:29:15,588 : INFO : topic #233 (0.002): 0.211*"Washington" + 0.163*

2019-12-06 18:29:57,697 : INFO : topic diff=0.041038, rho=0.102062
2019-12-06 18:29:57,715 : INFO : PROGRESS: pass 0, at document #194000/197377
2019-12-06 18:30:03,285 : INFO : merging changes from 2000 documents into a model of 197377 documents
2019-12-06 18:30:06,377 : INFO : topic #62 (0.002): 0.158*"visual" + 0.080*"eye" + 0.076*"Style" + 0.067*"Crow" + 0.048*"Rita" + 0.048*"vision" + 0.046*"resolution" + 0.037*"Visual" + 0.027*"eye." + 0.024*"Hat"
2019-12-06 18:30:06,379 : INFO : topic #261 (0.002): 0.080*"Railway" + 0.054*"train" + 0.048*"bus" + 0.040*"Young" + 0.036*"railway" + 0.036*"trains" + 0.031*"tracks" + 0.031*"Rail" + 0.031*"operated" + 0.030*"transport"
2019-12-06 18:30:06,380 : INFO : topic #406 (0.002): 0.320*"language" + 0.066*"languages" + 0.042*"language." + 0.031*"Language" + 0.026*"spoken" + 0.026*"Latin" + 0.020*"people" + 0.016*"also" + 0.015*"languages." + 0.014*"speak"
2019-12-06 18:30:06,382 : INFO : topic #25 (0.002): 0.467*"De" + 0.061*"Julius" + 0.057*"C

[(9,
  '0.112*"written" + 0.081*"names" + 0.056*"characters" + 0.054*"script" + 0.052*"letter" + 0.040*"letters" + 0.039*"alphabet" + 0.038*"writing" + 0.033*"used" + 0.033*"write"'),
 (293,
  '0.186*"Additional" + 0.127*"project" + 0.091*"Media" + 0.091*"Category" + 0.062*"Project" + 0.048*"file" + 0.042*"projects" + 0.038*"namespace" + 0.028*"1958." + 0.027*"exit"'),
 (396,
  '0.064*"Entertainment" + 0.055*"franchise" + 0.042*"DJ" + 0.042*"Top" + 0.041*"Kids" + 0.034*"Wild" + 0.034*"Morgan" + 0.033*"Hunt" + 0.033*"appearances" + 0.030*"featuring"'),
 (25,
  '0.446*"De" + 0.068*"Julius" + 0.050*"drove" + 0.050*"Caesar" + 0.043*"acquisition" + 0.035*"3000" + 0.035*"Utrecht" + 0.033*"merge" + 0.027*"Description" + 0.018*"\'11"'),
 (272,
  '0.209*"Virginia" + 0.149*"Bay" + 0.114*"Series" + 0.108*"Maryland" + 0.057*"Baltimore" + 0.033*"minutes" + 0.025*"seconds" + 0.024*"West" + 0.019*"Camden" + 0.017*"Herald"'),
 (196,
  '0.140*"Davis" + 0.042*"Make" + 0.040*"stub" + 0.040*"effort" + 0.0

In [247]:
tmp_fname = get_tmpfile("simple_wiki_lda_{}.model".format(NUM_TOPICS))
lda_model.save(tmp_fname) 

2019-12-06 18:30:30,548 : INFO : saving LdaState object under /tmp/simple_wiki_lda_500.model.state, separately None
2019-12-06 18:30:30,549 : INFO : storing np array 'sstats' to /tmp/simple_wiki_lda_500.model.state.sstats.npy
2019-12-06 18:30:30,680 : INFO : saved /tmp/simple_wiki_lda_500.model.state
2019-12-06 18:30:30,735 : INFO : saving LdaModel object under /tmp/simple_wiki_lda_500.model, separately ['expElogbeta', 'sstats']
2019-12-06 18:30:30,735 : INFO : storing np array 'expElogbeta' to /tmp/simple_wiki_lda_500.model.expElogbeta.npy
2019-12-06 18:30:30,854 : INFO : not storing attribute id2word
2019-12-06 18:30:30,855 : INFO : not storing attribute state
2019-12-06 18:30:30,855 : INFO : not storing attribute dispatcher
2019-12-06 18:30:30,856 : INFO : saved /tmp/simple_wiki_lda_500.model


In [248]:
lda_model.get_document_topics(gensim_tfidf_vectors[13])

[(27, 0.015680706),
 (46, 0.03095858),
 (50, 0.05212118),
 (58, 0.19012746),
 (116, 0.0119828405),
 (131, 0.03422434),
 (145, 0.03823592),
 (224, 0.022867618),
 (353, 0.15506709),
 (384, 0.014111323),
 (429, 0.23880689),
 (437, 0.09666438),
 (480, 0.026295368)]

In [250]:
lda_model.get_term_topics('Australia')

[(326, 0.019187158), (429, 0.10089274)]

In [253]:
lda_model.show_topic(429)

[('Australian', 0.110887766),
 ('Canada', 0.10234875),
 ('Australia', 0.1016037),
 ('Canadian', 0.07011058),
 ('Ontario', 0.036934573),
 ('Hockey', 0.034324437),
 ('Canada.', 0.031894933),
 ('Australia.', 0.028587492),
 ('Canadians', 0.02778754),
 ('Toronto', 0.02758234)]

In [256]:
lda_model.print_topic(326)

'0.296*"New" + 0.236*"Zealand" + 0.167*"Fribourg" + 0.031*"Toledo" + 0.029*"Zealand." + 0.025*"*East" + 0.022*"Dawson" + 0.019*"Australia" + 0.016*"Māori" + 0.014*"induced"'

In [257]:
lda_model.show_topics(5)

[(114,
  '0.206*"active" + 0.144*"Mount" + 0.089*"shield" + 0.057*"plate" + 0.034*"volcano" + 0.026*"volcanic" + 0.023*"eruption" + 0.022*"rising" + 0.015*"caused" + 0.015*"crater"'),
 (72,
  '0.397*"International" + 0.141*"Airport" + 0.076*"airport" + 0.069*"Airlines" + 0.035*"1980." + 0.027*"Delta" + 0.024*"connection" + 0.019*"airline" + 0.017*"Lines" + 0.017*"Airport."'),
 (256,
  '0.054*"Communities" + 0.051*"him." + 0.046*"her." + 0.031*"cast" + 0.029*"Jason" + 0.029*"kill" + 0.025*"Helen" + 0.025*"super" + 0.023*"beautiful" + 0.023*"girl"'),
 (471,
  '0.105*"University" + 0.075*"founded" + 0.064*"Institute" + 0.062*"university" + 0.049*"research" + 0.037*"located" + 0.031*"Technology" + 0.026*"suburb" + 0.025*"serves" + 0.024*"Sciences"'),
 (237,
  '0.394*"system" + 0.207*"central" + 0.072*"system." + 0.037*"Common" + 0.023*"Oct" + 0.018*"nervous" + 0.018*"3,000" + 0.017*"bold" + 0.017*"governmental" + 0.016*"route."')]